In [217]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    from zipfile import ZipFile
    
    response = urllib.request.urlretrieve(
        url,filename)
    #unzip
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

In [218]:
# define fuction to read data from xml file
def import_data(filename):
    import xml.etree.ElementTree as et 
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    return xroot;

In [219]:
# define function to create distance matrix
def dist_matrix(cities, xroot):
    #create distance matrix
    import numpy as np
    distance = np.zeros((cities,cities))
    
    #import data
    import xml.etree.ElementTree as et
    from_node = 0
    for child in xroot.iter('vertex'):
        for child1 in child:
            dist = float(child1.attrib.get('cost'))
            to_node = int(child1.text)
            distance[from_node, to_node] = dist

        from_node += 1
    
    max_distance = np.nanmax(distance)
    for i in range(cities):#
        distance[i,i] = max_distance*10 #very large number for distance to itself => no revisited 

    return distance 

In [220]:
# encoding a tour
def encode(detour):
    cities = len(detour)
    master = [x for x in range(cities)]
    entour = []
    
    for i in range(cities):
        entour.append(master.index(detour[i]))
        master.remove(detour[i])
    
    return entour

In [221]:
#decoding an individual
def decode(entour):
    cities = len(entour)    
    master = [x for x in range(cities)]
    detour = []
    
    for i in range(cities):
        detour.append(master[entour[i]])
        master.remove(master[entour[i]])
    
    return detour

In [222]:
# define function to swap 2 edges of nodes with index m and n (starting index 0)  
def tour_swap (tour, m, n):
    new_tour = copy.deepcopy(tour)
    new_tour[m:n+1] = reversed(new_tour[m:n+1])
    return new_tour

In [223]:
def two_opt (distance, tour):
    import copy
    
    best_tour = tour
    best_delta = 0
    improvement = True
    
    while improvement: 
        improvement = False
        for i in range(1,len(tour)-2):
            for j in range(i+1, len(tour)-1):
                temp_tour = tour_swap(tour, i, j)
                delta = totalcost(distance, tour[(i-1):(j+2)]) - totalcost(distance, temp_tour[(i-1):(j+2)]) #improvement = positive delta
                
                if delta > best_delta:
                    best_tour = copy.deepcopy(temp_tour)
                    best_delta = delta
                    improvement = True
    
    return best_tour, totalcost(best_tour)

In [224]:
# define function to calculate the length of the tour 
def totalcost(distance, tour):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]
    return length

In [225]:
#evaluate a tour
def evaluate(distance, tour):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]

    length += distance[tour[-1]][tour[0]] #return to starting point
  
    return length

In [226]:
#format of a solution
def create_solution(distance, entour, codetype = 'encode'):
    if codetype == 'encode':
        tour = decode(entour)
    elif codetype == 'permutation':
        tour = entour
    else:
        raise ValueError("Wrong codetype")

    solution = [evaluate(distance, tour), entour]
    
    return solution

In [227]:
# random initialize
def initialize(cities, distance, pop_size, codetype = 'encode'):
    solution_pop = []
    entour_pop = []
    
    while len(entour_pop) < pop_size: #iterate to get pop_size candidates 
        entour = []
        
        if codetype == 'encode':    
            for i in range(cities):
                entour.append(np.random.randint(0,cities-i))  #position <= # of remaning cities
        elif codetype == 'permutation':
            entour = np.arange(cities)
            np.random.shuffle(entour)
            entour = list(entour)
        else:
            raise ValueError("Wrong codetype")            
            
        if entour not in entour_pop:
            entour_pop.append(entour)
            solution_pop.append(create_solution(distance, entour, codetype))
    
    return solution_pop #return a population of solution candidates

In [228]:
# selection
def select(solution_pop, offspring_size, selecttype = 'fitness', tournament_size = None):
    offspring = []
    expected = np.mean([ sol[0] for sol in solution_pop])
    pop_size = len(solution_pop)
    f = [ (2*expected - solution_pop[i][0]) for i in range(pop_size)] #calculate f of each candidate
    sum_f = sum(f)

    if selecttype == 'tournament':
        #tournament size = # of solutions selected for each tournament
        if tournament_size is None: 
            tournament_size = len(solution_pop)//4
        
        for i in range(offspring_size):
            tournament = np.random.choice(np.arange(pop_size), size = tournament_size, replace = False) #random select k index
            tour_f = [f[i] for i in tournament]
            winner = tournament[np.argmax(tour_f)] #return index of winner
            offspring.append(solution_pop[winner])
            
        return offspring
    
    
    elif selecttype == 'rank': #rank selection
        f_index = [(value, idx) for (idx, value) in enumerate(f)]
        f_index = sorted(f_index, reverse = True) #sort based on value
        rank_index = [(idx, rank) for (rank, (value,idx)) in enumerate(f_index)] #get rank
        rank_index = sorted(rank_index) #sorted base on index
        rank = [r[1] for r in rank_index] #get a list of rank sorted on index
        
        eMax = 1 + np.random.random()
        eMin = 2 - eMax
        prob = [ (1/pop_size) * (eMax - (eMax - eMin)*(rank[i] - 1)/(pop_size-1)) for i in range(pop_size)] #individual prob
        
        prob_range = [ sum(prob[0:i+1]) for i in range(pop_size)] #cumulative prob of each candidate
        
    elif selecttype == 'fitness': #default selection mechanism
        prob_range = [ sum(f[0:i+1])/sum_f for i in range(pop_size)] #cumulative prob of each candidate
    
    else:
        raise ValueError("Wrong selecttype")        
    
    
    for i in range(offspring_size):
        p = np.random.random()

        for j in range(pop_size):
            if prob_range[j] >= p:
                offspring.append(solution_pop[j])
                break
        
    return offspring

In [229]:
#recombination and mutation
def recomb(cities, distance, solution_pop, offspring,
           recprob, muprob, crossprob = 0.5, mulocprob = 0.5, crosstype = 'uniform', codetype = 'encode'):

    pop = [s[1] for s in solution_pop] #set of decision
    new_solution_pop = []
    
    for k in range(len(offspring)//2): 

        #recombination
        if np.random.random() < recprob:
            child = []
            
            #crossover
            if crosstype == 'uniform': #uniform crossover
                for i in range(cities):
                    if np.random.random() < crossprob:
                        child.append(offspring[2*k][1][i]) #from offspring1
                    else: 
                        child.append(offspring[2*k+1][1][i]) #from offspring2
            
            elif crosstype == '1point': #1 point crossover
                cp=np.random.randint(1,cities-1)
                child = offspring[2*k][1][:cp] + offspring[2*k+1][1][cp:]
            
            elif crosstype == '2point': #2 point crossover
                cp1=np.random.randint(1,cities-2)
                cp2=np.random.randint(cp1+1,cities)
                child = offspring[2*k][1][:cp1] + offspring[2*k+1][1][cp1:cp2] + offspring[2*k][1][cp2:]
            else:
                raise ValueError("Wrong selecttype")
            
            
            #mutation
            if np.random.random() < muprob:
                if codetype == 'permutation':
                    (loc1, loc2) = np.random.choice(child, size = 2, replace = False)
                    temp = child[loc1]
                    child[loc1] = child[loc2]
                    child[loc2] = temp
                    
                elif codetype == 'encode':
                    for i in range(cities):
                        if np.random.random() < mulocprob:
                            child[i] = min(cities-1-i,child[i]+1)
                        else: child[i] = max(0,child[i]-1)
                else:
                    raise ValueError("Wrong codetype")


            #add to population
            if child not in pop:
                pop.append(child)
                new_solution_pop.append(create_solution(distance, child, codetype))
            
    return new_solution_pop

In [230]:
# replacement
def replace(solution_pop, new_solution_pop, pop_size, reptype = 'best', selecttype = None):
    pop = []
    
    if reptype == 'complete':
        return new_solution_pop
    
    elif reptype == 'best':
        total_pop = solution_pop + new_solution_pop
        sortedpop = sorted(total_pop) #sorted by cost of a solution
        pop = sortedpop[0:pop_size]
        
    elif reptype == 'select':
        total_pop = solution_pop + new_solution_pop
        pop = select(total_pop, pop_size, selecttype)
    
    return pop 

Problem specific
• Solution representation => encode vs permutation
• Evaluation of solutions: fitness => 2-opt
• Design of genetic operators recombination and crossover (uniform vs 1-point vs 2-point)

Generic parameters
• Population size
• Number of newly generated solutions in each iteration
• Selection mechanism (fitness or rank)
• Recombination and mutation probabilities
• Replacement strategy
• Termination criterion


In [231]:
import numpy as np
import pandas as pd
import math
import copy

In [232]:
# 120 cities
cities = 120

di = pd.read_excel("gr120.xlsx",sheet_name="DistanceMatrix")
distance=di.values

for i in range(cities):
    distance[i,i] = 10000

In [233]:
# Medium size data set: # of nodes = 152
url1 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr152.xml.zip'
zip1 = 'pr152.xml.zip'
file1 = 'pr152.xml'
cities1 = 152

load_data(url1, zip1)
xroot1 = import_data(file1)
distance1 = dist_matrix(cities1, xroot1)

In [234]:
# Larger size data set: # of nodes = 264
url2 = 'http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/XML-TSPLIB/instances/pr264.xml.zip'
zip2 = 'pr264.xml.zip'
file2 = 'pr264.xml'
cities2 = 264

load_data(url2, zip2)
xroot2 = import_data(file2)
distance2 = dist_matrix(cities2, xroot2)

In [235]:
#parameters
pop_size = 100
offspring_size = 500
iterations = 100
recombination = 0.9
crossover = 0.5
mutation = 0.3
loc_mutation = 0.3

In [248]:
codetype = 'encode'
selecttype = 'tournament'
crosstype = '2point'
reptype = 'best'

In [250]:
#main algorithm    
solution_pop = initialize(cities, distance, pop_size, codetype = codetype )

for it in range(iterations):
    offspring = select(solution_pop, offspring_size, selecttype = selecttype)
    new_solution_pop = recomb(cities, distance, solution_pop, offspring,
                          recprob = recombination, muprob = mutation, crossprob = crossover, mulocprob = loc_mutation, 
                          crosstype = crosstype, codetype = codetype)

    solution_pop = replace(solution_pop, new_solution_pop, pop_size, reptype = reptype)

In [251]:
best_solution = solution_pop[0]
print('Best objective value', best_solution[0])
print('Decision', best_solution[1])

Best objective value 43111
Decision [70, 94, 111, 62, 25, 111, 96, 27, 83, 96, 39, 90, 105, 58, 89, 68, 83, 86, 36, 63, 42, 10, 21, 61, 87, 54, 31, 43, 73, 26, 37, 3, 31, 67, 3, 54, 60, 30, 65, 52, 54, 74, 12, 14, 53, 19, 9, 69, 27, 21, 0, 63, 28, 59, 64, 53, 42, 58, 0, 6, 41, 31, 48, 50, 20, 0, 25, 2, 24, 41, 13, 32, 3, 8, 25, 32, 3, 12, 18, 34, 7, 0, 8, 19, 18, 21, 2, 10, 19, 10, 3, 4, 8, 19, 12, 11, 19, 5, 16, 2, 10, 14, 12, 9, 0, 8, 3, 0, 5, 4, 7, 7, 2, 2, 3, 1, 0, 0, 0, 0]


In [112]:
totalcost(decode([112, 17, 87, 0, 94, 0, 47, 96, 66, 21, 31, 39, 69, 84, 75, 48, 98, 16, 69, 70, 84, 19, 89, 73, 34, 28, 11, 7, 4, 55, 68, 51, 50, 48, 74, 4, 11, 15, 80, 52, 17, 15, 51, 25, 74, 57, 64, 29, 32, 40, 53, 66, 45, 16, 62, 63, 17, 42, 6, 18, 3, 34, 9, 17, 2, 0, 37, 48, 11, 12, 29, 11, 10, 9, 20, 6, 27, 18, 13, 27, 16, 38, 12, 9, 3, 13, 28, 25, 12, 13, 14, 13, 23, 1, 21, 24, 10, 6, 9, 20, 17, 15, 0, 12, 2, 8, 3, 6, 1, 4, 1, 6, 0, 0, 3, 1, 2, 2, 1, 0]))

53541

In [140]:
start = np.arange(cities)
entour = np.random.shuffle(start)
print(start)

[ 18  92  85 117  71   7  89  41   6 118  77  34  45  87  88  96 100 105
  30  48  74  19  59  73 108   5  83  51  60 109  31  46 112  58 119  61
  20  49  13  57  25  32  80   1  50 104  39  24 114  86  93  38  81 111
  29  43   8  63  65   4 101  79  27  67   3  75   9 113 116  17  94  68
  98  53 115   2  23  64  15  69  12  70  28  78 106  95 102  52 110  47
  62  91  55  37  10  97  14  33  42  84 107  22  11  36  66  54  16  56
  72  21   0  76  40  35  44  26  90  99 103  82]


In [214]:
help(np.random.randrange)

AttributeError: module 'numpy.random' has no attribute 'randrange'